<a href="https://colab.research.google.com/github/kaledai069/Crossword-Solver-A-Neural-Transformer-based-Approach/blob/master/First_pass_%26_Second_pass_Assessment_with_thenewyorker_theLAtimes_newsday.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/kaledai069/Crossword-Solver-A-Neural-Transformer-based-Approach
%cd Crossword-Solver-A-Neural-Transformer-based-Approach/
!bash inference_setup.sh

Cloning into 'Crossword-Solver-A-Neural-Transformer-based-Approach'...
remote: Enumerating objects: 2997, done.
remote: Counting objects: 100% (1697/1697), done.
remote: Compressing objects: 100% (1358/1358), done.
remote: Total 2997 (delta 402), reused 1626 (delta 335), pack-reused 1300
Receiving objects: 100% (2997/2997), 15.37 MiB | 19.24 MiB/s, done.
Resolving deltas: 100% (596/596), done.
/content/Crossword-Solver-A-Neural-Transformer-based-Approach
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

# Assessing croswords from The New Yorker, The LA Times & Newsday

### Important Imports

In [7]:
# the usuals
import datetime
import time
import matplotlib.pyplot as plt
import cv2
import pprint
import shutil
import os
import re
import json
import glob

from copy import deepcopy
from pprint import pprint
from matplotlib.gridspec import GridSpec

# from crossword solver
from solver.Crossword import Crossword
from solver.BPSolver import BPSolver
from utils import puz_to_json, fetch_nyt_crossword
from draw_grid import get_grid

### Paths to all required files:

In [4]:
# First pass model ingredients
model_path = "/content/gdrive/MyDrive/First Pass Model/distilbert_EPOCHs_7_COMPLETE.bin"
ans_path = "/content/gdrive/MyDrive/First Pass Model/new_answer_list.tsv"
dense_embd_path = "/content/gdrive/MyDrive/First Pass Model/distilbert_embeddings_v2_0.pkl"

# Second pass model ingredients
reranker_path = "/content/gdrive/MyDrive/Second Pass Model/t5_word_segmented_pair_2_epochs"
reranker_model_type = 't5-small'

In [5]:
def solve(crossword, m_path, a_path, d_path, r_path, reranker_m_type, max_candidate = 50000, score_threshold = 0.5):
    solver = BPSolver(
                          crossword,
                          model_path = m_path,
                          ans_tsv_path = a_path,
                          dense_embd_path = d_path,
                          reranker_path = r_path,
                          reranker_model_type = reranker_m_type,
                          max_candidates = max_candidate,
                          score_improvement_threshold = score_threshold
                      )

    solution = solver.solve(num_iters = 60, iterative_improvement_steps = 10)
    return solution

In [6]:
NEWSDAY_PUZ_FILES = "/content/gdrive/MyDrive/Full Crossword Data/newsday"
LATIMES_PUZ_FILES = "/content/gdrive/MyDrive/Full Crossword Data/the-LA-times"
NEWYORKER_PUZ_FILES = "/content/gdrive/MyDrive/Full Crossword Data/the_new_yorker"

In [31]:
def extract_date(file_path):
    match = re.search(r'\d{2}-\d{2}-\d{4}', file_path)
    return match.group() if match else ''

def get_all_puz_files_path(dir_name):
    puz_files = glob.glob(os.path.join(dir_name, "*.puz"))
    puz_files = [path for path in puz_files if '2023' in path]

    sorted_puz_files = sorted(puz_files, key = extract_date)
    return sorted_puz_files

def get_puzzle_dict(puz_path):
    date = puz_path.split('/')[-1].replace('.puz', '').replace('crossword_', '').replace("-", "/")

    puzzle = puz_to_json(puz_path)

    for dim in ['across', 'down']:
        for grid_num, CA_tuple in puzzle['clues'][dim].items():
            clue = CA_tuple[0].strip()
            answer = CA_tuple[1].strip()

            if len(clue) == 0 or len(answer) == 0:
                return None

            puzzle['clues'][dim][grid_num] = [clue, answer]

    return date, puzzle

In [32]:
all_outputs = []
for puz_path in get_all_puz_files_path(NEWSDAY_PUZ_FILES):
    date_str, puzzle = get_puzzle_dict(puz_path)

    try:
        crossword = Crossword(puzzle)

        start_time = time.time()
        output = solve(crossword, model_path, ans_path, dense_embd_path, reranker_path, "t5-small", 50000)
        end_time = time.time()

        time_taken = end_time - start_time
        all_outputs.append((date_str, output, time_taken))
        print("Total time taken: ", time_taken)

    except:
        print(f"An error occured for date: {date_str}")